<a href="https://colab.research.google.com/github/Phanitha09/Titanic-code/blob/main/drowsinesscode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install opencv-python dlib imutils scipy

In [6]:
from google.colab import files

# Upload your .dat file and video
uploaded = files.upload()

Saving vdo2.mp4 to vdo2.mp4
Saving shape_predictor_68_face_landmarks (1).dat to shape_predictor_68_face_landmarks (1).dat


In [7]:

# Install required packages
!pip install opencv-python dlib imutils scipy gTTS playsound==1.2.2

In [8]:
import cv2
import dlib
from scipy.spatial import distance
from imutils import face_utils
import numpy as np
from gtts import gTTS
from playsound import playsound
import os
import time

def calculate_ear(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def speak_warning(message, filename="warning.mp3"):
    tts = gTTS(text=message, lang='en')
    tts.save(filename)
    try:
        playsound(filename)
    except Exception as e:
        print("Audio playback failed:", e)
    time.sleep(1)
    if os.path.exists(filename):
        os.remove(filename)


In [10]:
# Load dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks (1).dat")  # Upload this file to Colab

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20
frame_counter = 0
alert_triggered = False

cap = cv2.VideoCapture("vdo2.mp4")  # Upload your video file to Colab

frame_output = []

if cap.isOpened():
    ret, frame = cap.read()
    if ret:
        height, width = frame.shape[:2]
    else:
        height, width = (0, 0)
else:
    print("Error: Could not open the video.")
    height, width = (0, 0)

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = calculate_ear(leftEye)
        rightEAR = calculate_ear(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        if ear < EAR_THRESHOLD:
            frame_counter += 1
            if frame_counter >= EAR_CONSEC_FRAMES:
                cv2.putText(frame, "DROWSINESS DETECTED!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if not alert_triggered:
                    speak_warning("Drowsiness detected. Please stay alert.")
                    alert_triggered = True
        else:
            frame_counter = 0
            alert_triggered = False

        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)

        # Simulate ML classifier (e.g., if EAR < 0.25, we say it's 'Drowsy')
        classification = "Drowsy" if ear < EAR_THRESHOLD else "Alert"
        cv2.putText(frame, f"ML Class: {classification}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

        # Simulate vehicle response
        if classification == "Drowsy":
            cv2.putText(frame, "ACC: Speed Reduced", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame, "LDWS: Stay in Lane", (10, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    frame = cv2.resize(frame, (480, 360))
    frame_output.append(frame)

cap.release()

output_filename = 'output_full_system.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, fourcc, 10, (480, 360))

for frame in frame_output:
    out.write(frame)

out.release()
print("✅ Full system video saved as output_full_system.mp4")

Audio playback failed: Namespace Gst not available
Audio playback failed: Namespace Gst not available
Audio playback failed: Namespace Gst not available
Audio playback failed: Namespace Gst not available
✅ Full system video saved as output_full_system.mp4


In [18]:
import cv2
import dlib
from scipy.spatial import distance
from imutils import face_utils
import numpy as np

# EAR calculation
def calculate_ear(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Load models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks (1).dat")

# Get eye landmarks
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# EAR Threshold
EAR_THRESHOLD = 0.28
EAR_CONSEC_FRAMES = 5
frame_counter = 0
drowsy = False  # Track drowsiness state

# Read video file
cap = cv2.VideoCapture("vdo2.mp4")
frame_output = []

# Get frame dimensions before the loop
if cap.isOpened():
    ret, frame = cap.read()
    if ret:
        height, width = frame.shape[:2]
    else:
        height, width = (0, 0)  # or some default values if video is empty.
        print("Error: Could not read a frame from the video.")
else:
    print("Error: Could not open the video.")
    height, width = (0, 0)  # or some default values

# Reset the video capture to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = calculate_ear(leftEye)
        rightEAR = calculate_ear(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        if ear < EAR_THRESHOLD:
            frame_counter += 1
            if frame_counter >= EAR_CONSEC_FRAMES:
                cv2.putText(frame, "DROWSINESS DETECTED!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if not drowsy:
                    print("Drowsy Driver Detected!")
                    drowsy = True  # Update the state to drowsy
        else:
            frame_counter = 0
            if drowsy:
                print("Driver is alert now!")
                drowsy = False  # Update the state to not drowsy

        cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)

    # Resize frame for speed
    frame = cv2.resize(frame, (480, 360))  # Ensure resizing after processing
    frame_output.append(frame)

cap.release()  # Release the video capture object after processing is complete

# Save the processed video as MP4
output_filename = 'output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*'mp4v'), 20, (480, 360))

for frame in frame_output:
    out.write(frame)

out.release()
print("✅ Video saved as output.mp4")


Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
Drowsy Driver Detected!
Driver is alert now!
✅ Video saved as output.mp4
